In [15]:
import pandas as pd
import numpy as np
import os 
import shutil
from tqdm import tqdm

import torch
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torch.utils.data import Dataset , DataLoader

import albumentations as A
import albumentations.pytorch

from efficientnet_pytorch import EfficientNet
    
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

import time
import cv2
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline  
%config InlineBackend.figure_format='retina'

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
# hyperparameter 
img_path = 'input/data/train/images'
batch_size = 128
num_workers = 2
learning_rate = 0.001
epochs = 10
img_size = 256

In [3]:
df = pd.read_csv('preprocessing_data.csv')
df.drop(columns=['Unnamed: 0'], inplace = True)
df.head()

,path,id,mask,gender,age,label
0,input/data/train/images/000001_female_Asian_45...,000001,0,1,1,4
1,input/data/train/images/000001_female_Asian_45...,000001,0,1,1,4
2,input/data/train/images/000001_female_Asian_45...,000001,0,1,1,4
3,input/data/train/images/000001_female_Asian_45...,000001,1,1,1,10
4,input/data/train/images/000001_female_Asian_45...,000001,2,1,1,16


In [6]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, path , df ,transform = None):
        
        self.path = path
        self.df = df
        self.transform = transform
        
    def __getitem__(self,idx):
        image = cv2.imread(self.df['path'].iloc[idx])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            augmented = self.transform(image=image) 
            image = augmented['image']
        label = self.df['label'].iloc[idx]
        return image, label

    def __len__(self):
        return len(self.df)

In [7]:
# albumentation 사용 
train_transforms = A.Compose([
    A.Resize(256,256),
    A.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225]),
    A.pytorch.transforms.ToTensor()
])

val_transforms = A.Compose([
    A.Resize(256,256),
    A.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225]),
    A.pytorch.transforms.ToTensor()
])

In [19]:
train_x , val_x , train_y , val_y = train_test_split(df , df['label'] , test_size=0.3 , shuffle = False)

train_dataset = CustomDataset(img_path , train_x , train_transforms)
val_dataset = CustomDataset(img_path , val_x , val_transforms)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size , shuffle = True , num_workers = num_workers)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size = batch_size , shuffle = True , num_workers = num_workers)

In [9]:
class BaseModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = EfficientNet.from_pretrained('efficientnet-b0')
        self.model._fc = nn.Linear(1280,18)
        
    def forward(self,x):
        x = self.model(x)
        return x

In [10]:
def train(epochs , train_loader, val_loader , model , criterion , optimizer , lr_scheduler):
    for epoch in range(epochs):
        ##################################### train ################################
        model.train()
        
        loss_train_sum = 0
        acc_train_sum = 0
        
        for i , (img , target) in enumerate(tqdm(train_loader)):
            img = img.to(device)
            target = target.to(device)
            
            y_pred = model.forward(img)
            loss = criterion(y_pred, target)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_train_sum += loss
            acc_train_sum += (y_pred.argmax(1) == target).sum().item()/ batch_size

        loss_train_avg = loss_train_sum / len(train_loader)
        acc_train_avg = acc_train_sum / len(train_loader)
        print(f" epoch:[{epoch+1}/{epochs}] cost:[{loss_train_avg:.3f}] acc : [{acc_train_avg : .3f}]")
        
        ##################################### eval ################################
        model.eval()
        
        loss_val_sum = 0
        acc_val_sum = 0
        
        for i , (img , target) in enumerate(tqdm(val_loader)):
            img = img.to(device)
            target = target.to(device)
            
            with torch.no_grad():
                y_pred = model.forward(img)
                loss = criterion(y_pred, target)
            
            loss_val_sum += loss
            acc_val_sum += (y_pred.argmax(1) == target).sum().item()/ batch_size
        
        loss_val_avg = loss_val_sum / len(val_loader)
        acc_val_avg = acc_val_sum / len(val_loader)
        print(f" epoch:[{epoch+1}/{epochs}] cost:[{loss_val_avg:.3f}] acc : [{acc_val_avg : .3f}]")
        
        lr_scheduler.step()
        

In [11]:
Model = BaseModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(Model.parameters() , lr = learning_rate)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, 5)

Loaded pretrained weights for efficientnet-b0


In [12]:
train(epochs , train_loader , val_loader , Model, criterion , optimizer , lr_scheduler)

  0%|          | 0/45 [00:00<?, ?it/s]

 epoch:[1/10] cost:[0.511] acc : [ 0.841]


  0%|          | 0/104 [00:00<?, ?it/s]

 epoch:[1/10] cost:[0.457] acc : [ 0.858]


  0%|          | 0/45 [00:00<?, ?it/s]

...

 epoch:[3/10] cost:[0.049] acc : [ 0.979]


  0%|          | 0/104 [00:00<?, ?it/s]

 epoch:[3/10] cost:[0.092] acc : [ 0.957]


 77%|███████▋  | 80/104 [01:17<00:23,  1.04it/s]


KeyboardInterrupt: 

In [ ]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

In [ ]:
test_dir = 'input/data/eval'

In [ ]:
img_size = 256
# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
transform = transforms.Compose([
                                    transforms.Resize((img_size, img_size)),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406],
                                                         [0.229, 0.224, 0.225])])
dataset = TestDataset(image_paths, transform)

loader = DataLoader(
    dataset,
    shuffle=False
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
Model.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in loader:
    with torch.no_grad():
        images = images.to(device)
        pred = Model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'submission.csv'), index=False)
print('test inference is done!')